In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from dotenv import load_dotenv, find_dotenv
import os

pd.set_option("display.max_columns", None)


In [2]:
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
FACEIT_PLAYERID = os.environ.get("FACEIT_PLAYERID")


In [3]:
def rename_cols(text: str):
    text = text.split(".")[1]
    text = text.split(" ")  # type: ignore
    text = "".join(list(map(str.capitalize, text)))
    return text


def read_json():
    with open(
        f"../data/raw/Game Stats/{FACEIT_PLAYERID}_games_stats.json", "r"
    ) as file:
        obj = json.load(file)
    df = pd.json_normalize(obj)
    df = df.rename(columns=rename_cols)
    return df

In [4]:
# df1 = read_json()
df1 = pd.read_pickle("./dataCompressed.pkl", compression="zip")
df2 = pd.read_pickle("./DataCompressedSteam.pkl", compression="zip")

In [5]:
type_casting = {
    "PlayerId": str,
    "CreatedAt": "datetime64[ns]",
    "Kills": int,
    "Deaths": int,
    "Assists": int,
    "Headshots": int,
    "FirstHalfScore": int,
    "SecondHalfScore": int,
    "QuadroKills": int,
    "TripleKills": int,
    "PentaKills": int,
    "Headshots%": int,
    "Mvps": int,
    "OvertimeScore": int,
    "Rounds": int,
}
df1 = df1.astype(type_casting, errors="ignore")
df1.dropna(how="any", inplace=True)
df1["Result"] = df1["Result"].astype(int).replace({1: "Win", 0: "Lose"})
df1["SteamData"] = False
df1["Score"] = df1["Score"].str.replace(" / ", ":")


In [6]:
common_columns = [
    "PlayerId",
    "Kills",
    "Assists",
    "Deaths",
    "Mvps",
    "Headshots%",
    "Score",
    "Map",
    "Result",
    "CreatedAt",
    "SteamData",
]


In [7]:
DF = pd.concat(
    [df1[common_columns], df2[common_columns]], join="inner", ignore_index=True
)
DF["Result"].replace({"True": "Won", "False": "Loss"}, inplace=True)

DF["Rounds"] = DF["Score"].str.split(":").apply(lambda x: int(x[0]) + int(x[1]))



In [8]:
DF["DateOrdinal"] = DF["CreatedAt"].apply(lambda date: date.toordinal())


In [9]:
rename_maps = {
    "Mirage": "de_mirage",
    "Cache": "de_cache",
    "Dust II": "de_dust2",
    "Overpass": "de_overpass",
    "Inferno": "de_inferno",
    "Cobblestone": "de_cbble",
    "Train": "de_train",
    "Office": "de_office",
    "Nuke": "de_nuke",
}

DF["Map"].replace(rename_maps, inplace=True)
DF.query("Map != 'de_office'", inplace=True)
DF.reset_index(drop=False, inplace=True)
DF.sort_values(by = "CreatedAt", ascending=True, inplace=True)
DF.reset_index(drop=True, inplace=True)


DF["Year"] = DF["CreatedAt"].dt.year
DF["Month"] = DF["CreatedAt"].dt.month
DF["Day"] = DF["CreatedAt"].dt.day


DF.query("Map != 'de_office'", inplace=True)
DF.sort_values(by = "CreatedAt", ascending=True, inplace=True)


In [12]:
DF.to_pickle("./AllDatabz2.pkl", compression="bz2")